# Query Sampling
This document provides an introduction to LDA Query Sampling. Query sampling is a technique for fitting a new document into an already existing topic model. 

This feature is still in beta testing, but the notebook provides a guideline for using it with your own texts.

In [1]:
%pylab inline

from vsm import *
from corpus import *
from collections import defaultdict
import json

Populating the interactive namespace from numpy and matplotlib
[20, 40, 60, 80]
/var/inphosemantics/darwin-2015/models/corpus-nltk-svdeenfrnlit-freq5-N12000-LDA-K{0}-book-500.npz
Loading LDA data from /var/inphosemantics/darwin-2015/models/corpus-nltk-svdeenfrnlit-freq5-N12000-LDA-K20-book-500.npz
Loading LDA data from /var/inphosemantics/darwin-2015/models/corpus-nltk-svdeenfrnlit-freq5-N12000-LDA-K40-book-500.npz
Loading LDA data from /var/inphosemantics/darwin-2015/models/corpus-nltk-svdeenfrnlit-freq5-N12000-LDA-K60-book-500.npz
Loading LDA data from /var/inphosemantics/darwin-2015/models/corpus-nltk-svdeenfrnlit-freq5-N12000-LDA-K80-book-500.npz
572
572
503 30
498


In [2]:
# global presentation settings
k = 20
m = lda_m[k]
v = lda_v[k]

In [3]:
from vsm.extensions.corpusbuilders import toy_corpus, corpus_from_strings
def build_sample(bookfile):
    print "opening corpus"
    with open(bookfile) as origin:
        origintext = [origin.read()]
    print "building corpus"
    origin = corpus_from_strings(origintext, nltk_stop=True, stop_freq=0)
    #origin = toy_corpus('origin1st.txt', is_filename=True, stop_freq=0, nltk_stop=True)
    origin.context_types = ['book']
    
    print "aligning corpus"
    c = align_corpora(v.corpus, origin)
    q = LdaCgsQuerySampler(v.model, old_corpus=v.corpus, new_corpus=c, context_type='book', align_corpora=False)
    q.train(n_iterations=200)
    return q

def get_topics(query_sample):
    return np.squeeze(query_sample.top_doc / sum(query_sample.top_doc))

In [4]:
origin_tops = get_topics(build_sample('data/origin-1e.txt'))

opening corpus
building corpus
aligning corpus
Begin LDA training for 200 iterations

100%|#########################################################################|


In [5]:
# some manipulations to force indexical order on the topics so the same topic is being compared across both documents
tops = v.aggregate_doc_topics(v.labels)
tops = np.array(tops[tops['i'].argsort()]['value'])

In [6]:
# comparing the new text to the old texts
from vsm.spatial import *
JS_dist(origin_tops, tops)

0.76216265585580867